In [230]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import mannwhitneyu
from scipy.stats import pearsonr, spearmanr

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
import warnings
import re
warnings.filterwarnings('ignore')
%matplotlib inline

In [264]:
df = pd.read_csv("genes/combined-gene-on-off.csv")
df["inr_effect"] = df["Sum of initiator effect"]
for motif in ["NRF1", "ZNF143", "NFY", "ETS", "CREB", "SP"]:
    df[motif] = df[f'{motif} +'] + df[f'{motif} -']

df_motif = df
df.shape, df_motif.shape

((83, 53), (83, 53))

In [265]:
tmp = df[["gene", "TATA_group", "TATA +"]]
tmp = tmp[tmp["TATA_group"] != "unknown"].drop_duplicates()
tmp[f'TATA + group'] = tmp.apply(lambda row: f'high TATA +' if float(row["TATA +"]) > 1 else f'low TATA +', axis=1)

contingency_table = pd.crosstab(tmp['TATA_group'], tmp['TATA + group'])
contingency_table


TATA + group,high TATA +,low TATA +
TATA_group,,
with_TATA,2,5
without_TATA,1,53


In [266]:
from scipy.stats import chi2_contingency
chi2, p, dof, ex = chi2_contingency(contingency_table)
print(f"Chi-squared: {chi2}, P-value: {p}, Degrees of freedom: {dof}")
print("Expected frequencies:")
print(ex)

from scipy.stats import fisher_exact
# Perform Fisher's exact test
odds_ratio, p_value = fisher_exact(contingency_table, alternative='two-sided')

print(f"Odds ratio: {odds_ratio}, P-value: {p_value}")

Chi-squared: 4.609640120415984, P-value: 0.031792704642467376, Degrees of freedom: 1
Expected frequencies:
[[ 0.3442623  6.6557377]
 [ 2.6557377 51.3442623]]
Odds ratio: 21.2, P-value: 0.03248124479021951


In [267]:
tmp[["TATA_group", "TATA + group"]].value_counts()

TATA_group    TATA + group
without_TATA  low TATA +      53
with_TATA     low TATA +       5
              high TATA +      2
without_TATA  high TATA +      1
dtype: int64

In [268]:
df_off = pd.DataFrame(columns=['gene', 'celltype', 'off-period'])
for index, row in df_motif.iterrows():
    gene = row['gene']
    celltype = row['celltype']

    # Extract off-period values from the string
    off_periods = [int(period) for period in re.findall(r'\d+', row['off-period'])]
    
    # Append each off-period to the new DataFrame
    for off_period in off_periods:
        if off_period >= 2:
            df_off = df_off.append({'gene': gene, 'celltype': celltype, 'off-period': off_period}, ignore_index=True)


df_off['off-period'] = df_off['off-period'].astype(int)
df_off = pd.merge(df_off, df.drop('off-period', axis=1), on=["gene", "celltype"])
df_off.shape

(7105, 53)

In [269]:
df_off.celltype.value_counts()

HBEC    5347
H9D3    1157
H9D0     601
Name: celltype, dtype: int64

In [270]:
df_off.head(1)

,gene,celltype,off-period,on-period,off-mean,on-mean,K-off-rate,K-on-rate,off-median,on-median,off-period-counts,on-period-counts,TATA_group,inr_group,chr_x,strand_x,TSS,geneID,TATA +,YY1 +,SP +,SP -,ETS +,ETS -,NFY +,NFY -,CREB +,CREB -,NRF1 +,NRF1 -,ZNF143 +,ZNF143 -,U1 snRNP +,chr_y,start,end,strand_y,gene_id,transcript_id,distance2tss,Sum of initiator effect,strand,H3K27me3_tss,cluster,gene_type,comment,inr_effect,NRF1,ZNF143,NFY,ETS,CREB,SP
0,LUZP1,HBEC,213,"[7, 7, 2]",79.75,5.333333,0.1875,0.012539,47.5,7.0,4,3,without_TATA,without_inr,chr1,-,23178121.0,ENSG00000169641.9,0.009,0.011,-0.126,-0.16,-0.028,-0.07,0.045,0.065,0.498,0.435,-0.004,0.001,-0.001,-0.067,0.186,chr1,23084023.0,23177808.0,-,ENSG00000169641,NaN,313.0,1.355976,-,0.0,cluster0,simple,NaN,1.355976,-0.003,-0.068,0.11,-0.098,0.933,-0.286


In [271]:
df_motif[df_motif["NRF1"] > 1][["gene", "NRF1", "off-mean"]].sort_values("gene")

,gene,NRF1,off-mean
30,EEF1D,1.455,22.916667
42,EEF1D,1.455,17.040816
57,EEF1D,1.455,22.300000
12,RABL6,1.270,24.900000
50,SEC16A,2.808,21.490494
66,SEC16A,2.808,18.414414
55,SNURF,1.830,28.800000
69,WDR4,1.851,69.500000
51,XRCC5,3.105,19.333333
60,XRCC5,3.105,21.700000


In [272]:
df_motif.head(1)

,gene,off-period,on-period,off-mean,on-mean,K-off-rate,K-on-rate,off-median,on-median,off-period-counts,on-period-counts,TATA_group,inr_group,chr_x,strand_x,TSS,geneID,TATA +,YY1 +,SP +,SP -,ETS +,ETS -,NFY +,NFY -,CREB +,CREB -,NRF1 +,NRF1 -,ZNF143 +,ZNF143 -,U1 snRNP +,chr_y,start,end,strand_y,gene_id,transcript_id,distance2tss,Sum of initiator effect,celltype,strand,H3K27me3_tss,cluster,gene_type,comment,inr_effect,NRF1,ZNF143,NFY,ETS,CREB,SP
0,LUZP1,"[213, 55, 11, 40]","[7, 7, 2]",79.75,5.333333,0.1875,0.012539,47.5,7.0,4,3,without_TATA,without_inr,chr1,-,23178121.0,ENSG00000169641.9,0.009,0.011,-0.126,-0.16,-0.028,-0.07,0.045,0.065,0.498,0.435,-0.004,0.001,-0.001,-0.067,0.186,chr1,23084023.0,23177808.0,-,ENSG00000169641,NaN,313.0,1.355976,HBEC,-,0.0,cluster0,simple,NaN,1.355976,-0.003,-0.068,0.11,-0.098,0.933,-0.286


In [273]:
df_off[df_off.inr_effect.isnull()]["gene"].unique()

array(['MIR222HG', 'CA5AP1'], dtype=object)

In [ ]:


xcols = [ 'TATA +', 'YY1 +',
         'NRF1', 'NRF1 +', 'NRF1 -',
         'ZNF143', 'ZNF143 +', 'ZNF143 -',
         'NFY', 'NFY +', 'NFY -',
         'ETS', 'ETS +', 'ETS -',
         'CREB', 'CREB +', 'CREB -',
         'SP', 'SP +', 'SP -', 
         'U1 snRNP +',
        'inr_effect']

xcols = [ 'TATA +', 'YY1 +',
         'NRF1', 
         'ZNF143', 
         'NFY',
         'ETS',
         'CREB', 
         'SP', 
         'inr_effect']


motif_threshold = {}
for xcol in xcols:
    motif_threshold[xcol] = 1
    
res = []
for xcol in xcols:
    plt.figure(figsize=(8, 6))
    data = df_motif[~df_motif[xcol].isnull()].copy()
    
    if xcol == 'TATA +':
        sns.scatterplot(data=data, x=xcol, y='off-mean', hue='TATA_group', 
                        palette={'with_TATA': 'red', 'without_TATA': 'blue', 'unknown': 'grey'})
    else:
        sns.scatterplot(data=data, x=xcol, y='off-mean')
    
    plt.title(f'{xcol} vs off-mean')
    plt.xlabel(xcol)
    plt.ylabel('off-mean')
    
    # Calculate Pearson and Spearman correlation coefficients
    pearson_corr, _ = pearsonr(data[xcol], data['off-mean'])
    spearman_corr, _ = spearmanr(data[xcol], data['off-mean'])
    
    # Calculate counts of motif-high and motif-low
    motif_high_count = data[data[xcol] > motif_threshold.get(xcol, 0)]["gene"].nunique()
    motif_low_count = data[data[xcol] <= motif_threshold.get(xcol, 0)]["gene"].nunique()
    
    # Annotate the plot with correlation coefficients and counts
    plt.annotate(f"Pearson: {pearson_corr:.2f}\nSpearman: {spearman_corr:.2f}\nhigh {xcol} genes: {motif_high_count}\nlow {xcol} genes: {motif_low_count}", 
                 xy=(0.05, 0.95), 
                 xycoords='axes fraction', 
                 fontsize=12, 
                 ha='left', 
                 va='top')
    if xcol in motif_threshold:
        threshold = motif_threshold[xcol]
        plt.axvline(x=threshold, color='r', linestyle='--', label='Threshold')
    
    plt.legend()
    plt.show()
    
    ##########################################################################################
    col = 'off-period'
    data = df_off[~df_off[xcol].isnull()].copy()
    title = f'Distribution of off-period -- combine HEBC/H9D0/H9D3'
    
    order = [f'high_{xcol}', f'low_{xcol}']
    data[f'{xcol}_group'] = data.apply(lambda row: f'high_{xcol}' if row[xcol] > motif_threshold[xcol]
                                       else f'low_{xcol}', axis=1)
    
    

    plt.figure(figsize=(10, 6))
    sns.boxplot(x=f'{xcol}_group', y=col, data=data, palette="Set2", order=order)
    plt.title(f'motif:{xcol} {title}')
    plt.ylabel(col);
    mean_values = data.groupby(f'{xcol}_group')[col].median()
    gene_counts = data.groupby(f'{xcol}_group')['gene'].nunique()
    off_period_counts = data.groupby(f'{xcol}_group').size()
    new_labels = [f'{o} (n-genes={gene_counts[o]}, n-off-period={off_period_counts[o]})' for o in order]
    
    plt.xticks(ticks=range(len(new_labels)), labels=new_labels)
    for i, label in enumerate(order):
        plt.text(i, mean_values[label], f'{mean_values[label]:.2f}', ha='center', va='bottom', color='red')

    d1 = data[data[f'{xcol}_group'] == f'high_{xcol}'][col]
    d2 = data[data[f'{xcol}_group'] == f'low_{xcol}'][col]
    stat, p = mannwhitneyu(d1, d2)
    
    pos_genes = data[data[f'{xcol}_group']== f'high_{xcol}']["gene"].unique()
    res.append([xcol, 
                gene_counts[f'high_{xcol}'],  gene_counts[f'low_{xcol}'],
                off_period_counts[f'high_{xcol}'],  off_period_counts[f'low_{xcol}'],
                mean_values[f'high_{xcol}'],  mean_values[f'low_{xcol}'],
                p,
                ','.join(pos_genes)
               ])
    
    p = f'p = {p:.3e}' if p < 0.001 else f'p = {p:.4f}'
    
    # Annotate plot with the significance level
    x1, x2 = 0, 1  # x coordinates for the two categories you're comparing
    y, h, color = data[col].max() , 1, 'k'  # y position and height for the line, and color
    plt.plot([x1, x2], [y, y], lw=1.5, c=color)
    plt.text((x1+x2)*.5, y, p, ha='center', va='bottom', color=color)

    plt.show()
        
    


In [283]:
from statsmodels.stats.multitest import multipletests

df_res = pd.DataFrame.from_records(res)
df_res.columns = ["motif",
                  "number of high group genes",
                  "number of low group genes",
                  "number of high group off-period",
                  "number of low group off-period ",
                  "high group off-period median",
                  "low group off-period median",
                  "pvalue",
                  "genes"
               ]

p_values = df_res["pvalue"]

_, fdr_corrected_p_values, _, _ = multipletests(p_values, method='fdr_bh')
df_res['FDR'] = fdr_corrected_p_values
def format_value(value):
    if value < 0.001:
        return f"{value:.2e}"
    return value
df_res["pvalue"] = df_res["pvalue"].apply(format_value)
df_res['FDR'] = df_res['FDR'].apply(format_value)
df_res

,motif,number of high group genes,number of low group genes,number of high group off-period,number of low group off-period,high group off-period median,low group off-period median,pvalue,genes,FDR
0,TATA +,4,61,585,6505,12.0,16.0,2.89e-08,"EEF2,TPM4,ERRFI1,SPTAN1",2.60e-07
1,YY1 +,5,60,99,6991,22.0,15.0,0.034796,"SAP30BP,RABL6,GCN1,DIDO1,DNAAF5",0.052195
2,NRF1,6,59,726,6364,13.0,15.0,2.11e-05,"RABL6,EEF1D,SEC16A,XRCC5,SNURF,WDR4",6.34e-05
3,ZNF143,5,60,432,6658,17.0,15.0,0.083896,"IPO7,SRRM2,RHOA,ITCH,THRAP3",0.107867
4,NFY,16,49,2602,4488,17.0,14.0,2.45e-07,"SAP30BP,STIP1,SND1,GDI2,RNF4,ERRFI1,ZMYND8,HNR...",1.10e-06
5,ETS,9,56,384,6706,18.0,15.0,0.01358,"SAP30BP,YTHDF3,RNF4,EIF4G1,PRRC2B,GCN1,ITCH,XR...",0.024445
6,CREB,9,56,650,6440,15.0,15.0,0.610027,"EEF2,GNB1,EIF4G1,SEPTIN2,IVNS1ABP,ZNF146,WDR4,...",0.686281
7,SP,20,45,1356,5734,14.0,16.0,0.010228,"B3GNT5,BNC1,GDI2,TPM3,RNF4,EEF2,TPM4,JUP,EEF1D...",0.023014
8,inr_effect,18,47,1219,5871,15.0,15.0,0.78694,"LUZP1,B3GNT5,STIP1,CAPRIN1,BNC1,GDI2,IPO7,YTHD...",0.78694


In [284]:
df_res.to_csv("tmp.csv")

In [160]:
df = pd.read_csv("genes/combined-gene-on-off.csv")
for motif in ["NRF1", "ZNF143", "NFY", "ETS", "CREB", "SP"]:
    df[motif] = df[f'{motif} +'] + df[f'{motif} -']

df_motif = df[~df['TATA +'].isnull()]
df.shape, df_motif.shape

((95, 51), (87, 51))

In [161]:





cols = [ 'TATA +', 'YY1 +',
         'NRF1', 'NRF1 +', 'NRF1 -',
         'ZNF143', 'ZNF143 +', 'ZNF143 -',
         'NFY', 'NFY +', 'NFY -',
         'ETS', 'ETS +', 'ETS -',
         'CREB', 'CREB +', 'CREB -',
         'SP', 'SP +', 'SP -', 
         'U1 snRNP +']

for col in cols:
    
    df[f'{col}'] = df.apply(lambda row: f'high' if row[col] > 1 else f'low', axis=1)


In [162]:
cols = ['gene', 'celltype',
       'TATA +', 'YY1 +',
         'NRF1', 'NRF1 +', 'NRF1 -',
         'ZNF143', 'ZNF143 +', 'ZNF143 -',
         'NFY', 'NFY +', 'NFY -',
         'ETS', 'ETS +', 'ETS -',
         'CREB', 'CREB +', 'CREB -',
         'SP', 'SP +', 'SP -', 
         'U1 snRNP +']
df[cols].to_csv("motif-group.csv", index=False)